In [ ]:
import time
from typing import Callable

import anywidget
from fastcore.test import *
from IPython.core.display import HTML
from olio.basic import empty
from olio.common import setup_console
from olio.widget import cleanupwidgets


In [ ]:
import os
os.environ['DEBUG_BRIDGET'] = 'True'

In [ ]:
from bridget.bridge_widget import anysource
from bridget.bridge_widget import BlockingMixin, blocks, ablocks
from bridget.helpers import kounter


In [ ]:
console, cprint = setup_console(140)

----

In [ ]:
print(13)
print(14)

13
14


In [ ]:
dh1 = display(HTML('asdf'), display_id=True)
dh2 = display(HTML('ghjk'), display_id=True)

In [ ]:
dh1.update(HTML('qwer'))

In [ ]:
dh2.update(HTML('tyui'))

In [ ]:
_xx = ['-_-', 'o_o']
def _show(finish: bool=False):
    if finish: print('._.')
    else: print(_xx[0], end="\r", flush=True); _xx[:] = _xx[::-1]

In [ ]:
class Cnt:
    def __init__(self, n=10, sleep=0.05): self.n, self.sleep, self.cnt = n, sleep, 0
    def __call__(self):
        self.cnt += 1
        time.sleep(self.sleep)
        return self.cnt > self.n-1


In [ ]:
done = blocks(cntr := Cnt(10), n=20, show=_show)
test_eq(cntr.cnt, 10)

._.


In [ ]:
done = await ablocks(cntr := Cnt(10, 0.2), show=_show)
if done: test_eq(cntr.cnt, 10)
else: print('timeout')

._.
timeout


In [ ]:
class BlockingWidget(anywidget.AnyWidget, BlockingMixin):
    _esm = anysource('// debugger;', '''
    export default { 
        async initialize({ model }) {
            function on_msg(msg) {
                console.log(`Received message:`, msg);
                if (!msg?.timeout) return model.send({ msg_id: msg.msg_id, response: 'no timeout' });
                (function loop(n) {
                    setTimeout(() => {
                        n += 100
                        if (n > msg.timeout*1000) {
                            console.log(`Sending response for msg_id:`, msg.msg_id);
                            model.send({ msg_id: msg.msg_id, response: 'done waiting', error: null });
                            return;
                        }
                        // console.log('.');
                        loop(n);
                    }, 100);
                })(0);
            }
            model.on("msg:custom", on_msg);
            setTimeout(() => {
                console.log(`Initialized.`);
                model.set('_loaded', true); model.save_changes();
            }, 1000);
        }
    };
    ''')

    def test_send(self, msg, buffers=None, *, timeout: float|None=None, 
            sleep: float = 0.1, n: int = 10, show: Callable[[bool], None]|None = None):
        self._start_time = time.time()
        print(f'Sending message {msg}...')
        res = self.send(msg, buffers, timeout=timeout, sleep=sleep, n=n, show=show)
        return None if res == (empty, empty) else res
    
    async def test_asend(self, msg, buffers=None, *, timeout: float=5.0, 
        sleep: float = 0.1, n: int = 10, show: Callable[[bool], None]|None = None
    ):
        self._start_time = time.time()
        print(f'Sending message {msg}...')
        res = await self.asend(msg, buffers, timeout=timeout, sleep=sleep, n=n, show=show)
        return None if res == (empty, empty) else res

    def __init__(self, *args, **kwargs):
        # self.setup_init_on_msg(self._handle_message)
        self.on_msg(self._handle_message)
        super().__init__(*args, **kwargs)
        print('Initializing...')

    def _handle_message(self, _, msg, buffers):
        e = time.time()
        self._last_message = e, msg
        print(f'\nelapsed: {e-self._start_time:.3f}s Received message: {msg}, buffers: {buffers}')

In [ ]:
cleanupwidgets('w')

w = BlockingWidget.create(show=_show, sleep=0.1)
print(f'loaded={w.loaded()}')
test_eq(w.loaded(), True)

Initializing...
._.
loaded=True


In [ ]:
idx = kounter('blocking')
a = w.test_send({'msg_id': idx, 'timeout': 2}, timeout=1, show=_show)
print(f"{idx=} ->", f"Timeout {time.time()-w._start_time:3f}" if not a else f"{a=}")
# test_is(a, None)

Sending message {'msg_id': 9, 'timeout': 2}...
._.
idx=9 -> Timeout 1.118046


In [ ]:
idx = kounter('blocking')
a = w.test_send({'msg_id': idx, 'timeout': 2}, timeout=3, show=_show)
print(f"{idx=} ->", f"Timeout {time.time()-w._start_time:3f}" if not a else f"{a=}")
# test_eq(a[0]['msg_id'], idx)  # type: ignore

Sending message {'msg_id': 10, 'timeout': 2}...
._.
idx=10 -> a=({'msg_id': 10, 'response': 'done waiting', 'error': None}, [])


In [ ]:
# cell 18
idx = kounter('blocking')
a = await w.test_asend({'msg_id': idx, 'timeout': 2}, timeout=1, show=_show)


Sending message {'msg_id': 11, 'timeout': 2}...
o_o

In [ ]:
print(f"{idx=} ->", f"Timeout {time.time()-w._start_time:3f}" if not a else f"{a=}")
# test_is(a, None)

idx=11 -> Timeout 1.078250


In [ ]:
# cell 20

idx = kounter('blocking')
a = await w.test_asend({'msg_id': idx, 'timeout': 2}, timeout=3, show=_show)


Sending message {'msg_id': 12, 'timeout': 2}...
._.


In [ ]:
print(f"{idx=} ->", f"Timeout {time.time()-w._start_time:3f}" if not a else f"{a=}")
# test_eq(a[0]['msg_id'], idx)  # type: ignore

idx=12 -> a=({'msg_id': 12, 'response': 'done waiting', 'error': None}, [])


In [ ]:
# cell 22
w.close()